In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
from selenium import webdriver
import time
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# driver = webdriver.Chrome()
# url = 'https://www.immoweb.be/en/search/apartment/for-rent?countries=BE&page=1&orderBy=newest'
# driver.get(url)
# time.sleep(3)
# try:
#     assert '<h1 class="title--2--dark page-error__title"> Unfortunately, something went wrong.</h1>' not in driver.page_source
#     soup = BeautifulSoup(driver.page_source, 'lxml')
#     set_urls = set()
#     for image in soup.find_all("a", attrs={"class": "card__title-link"}):
#         set_urls.add(image.get("href"))
# except:
#     driver.close()

In [3]:
def scrape_ad(driver, url):
    driver.get(url)
    return pd.read_html(driver.page_source)

In [13]:
driver = webdriver.Chrome()

# load dataframe from existing csv file or build new one from scratch
try:
    df = pd.read_csv('data/dump_df.csv').set_index('Unnamed: 0')
except:
    df = pd.DataFrame()

# set page number
n = 1
while True:
    # open page n and scrape it
    driver.get(f'https://www.immoweb.be/en/search/apartment/for-rent?countries=BE&page={n}&orderBy=newest')
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'lxml')

    # collect all the links to the ads
    for ad_link in soup.find_all("a", attrs={"class": "card__title-link"}):
        data = None
        url = ad_link.get("href")
        url = url.split('?')[0] # split url and keep ad unique id

        # scrape ad from url if url not already in the final table
        if url in df.index:
            continue
        try:
            data = scrape_ad(driver, url)
        except:
            time.sleep(3)
            try:
                data = scrape_ad(driver, url)
            except:
                print(f'Failed to scrape this ad: {url}')
        
        # transform scraped data and add to dataframe
        if data is not None:
            data = pd.concat(data, axis=0).reset_index(drop=True)
            data = data.set_index(0)
            data.columns = [url]
            data = data[~data.index.duplicated(keep='first')]
            data = data.T
            columns = [col for col in data.columns if isinstance(col, str) and 'Insure this property' not in col]
            df = pd.concat([df, data[columns]])

    # store dataframe in csv file
    df.to_csv('data/dump_df.csv')

    # increment page number
    n += 1

    # for testing on a determined number of pages
    if n == 3:
        break

driver.close()
df.shape

(60, 83)